# Описание проекта

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Откройте и изучите файл

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
df = pd.read_csv('users_behavior.csv')

In [32]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.6 KB


# Разбейте данные на выборки

In [35]:
# сохраним в переменную псевдорандомное число
rnd_st = 12345

In [42]:
# Поделим наш датасет на части
# df_train = 60%, df_valid и df_test = 20%
# Тем самым наши выборки будут поделены три части 3:1:1
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=rnd_st)
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=rnd_st)

# Исследуйте модели

In [95]:
# Подготовим тренировочную и валидационную выборки
train_features = df_train.drop(['is_ultra'], axis=1)
train_target = df_train['is_ultra']
valid_features = df_valid.drop(['is_ultra'], axis=1)
valid_target = df_valid['is_ultra']

In [96]:
# Напишем функцию, которая расчитывает правильность на валидационной выборке
def valid_accuracy_score(model):
    valid_predictions = model.predict(valid_features)
    return accuracy_score(valid_target, valid_predictions)

# Decision tree

In [97]:
# Узнаем оптимальную глубину дерева решений
for depth in range(1, 11):
    model = DecisionTreeClassifier(max_depth=depth, random_state=rnd_st)
    model.fit(train_features, train_target)
    print('max_depth = {}: {}'.format(depth, valid_accuracy_score(model)))

max_depth = 1: 0.7542768273716952
max_depth = 2: 0.7822706065318819
max_depth = 3: 0.7853810264385692
max_depth = 4: 0.7791601866251944
max_depth = 5: 0.7791601866251944
max_depth = 6: 0.7838258164852255
max_depth = 7: 0.7822706065318819
max_depth = 8: 0.7791601866251944
max_depth = 9: 0.7822706065318819
max_depth = 10: 0.7744945567651633


По результатам перебора видно, что значение max_depth=3 имеет лучший результат

# Random forest

In [143]:
for estim in range(1, 11):
    for depth in range(1, 11):
        model = RandomForestClassifier(n_estimators=estim, max_depth=depth, random_state=rnd_st)
        model.fit(train_features, train_target)
        print('estim = {:<5} depth = {:<5} {}'.format(estim, depth, valid_accuracy_score(model)))

estim = 1     depth = 1     0.7542768273716952
estim = 1     depth = 2     0.7853810264385692
estim = 1     depth = 3     0.7853810264385692
estim = 1     depth = 4     0.7744945567651633
estim = 1     depth = 5     0.776049766718507
estim = 1     depth = 6     0.7853810264385692
estim = 1     depth = 7     0.7776049766718507
estim = 1     depth = 8     0.7776049766718507
estim = 1     depth = 9     0.7791601866251944
estim = 1     depth = 10    0.7729393468118196
estim = 2     depth = 1     0.7371695178849145
estim = 2     depth = 2     0.7822706065318819
estim = 2     depth = 3     0.7853810264385692
estim = 2     depth = 4     0.7776049766718507
estim = 2     depth = 5     0.7729393468118196
estim = 2     depth = 6     0.7853810264385692
estim = 2     depth = 7     0.7776049766718507
estim = 2     depth = 8     0.7807153965785381
estim = 2     depth = 9     0.7667185069984448
estim = 2     depth = 10    0.7698289269051322
estim = 3     depth = 1     0.7465007776049767
estim = 3     

Решил сделать такой перебор для двух значений. Да, список длинный, но за то у нас теперь есть более лучший результат. В сравнении с Decision tree, Random forest с n_estimators=8, max_depth=8 даёт результат в 0.8

# Logistic regression

In [107]:
def logistic_regression():
    model = LogisticRegression(random_state=rnd_st)
    model.fit(features_train, target_train)
    return valid_accuracy_score(model)

In [108]:
logistic_regression()

0.7107309486780715

Здесь у нас худший результат. Выберем Random forest

# Проверьте модель на тестовой выборке

In [114]:
test_features = df_test.drop(['is_ultra'], axis=1)
test_target = df_test['is_ultra']

In [125]:
model = RandomForestClassifier(n_estimators=8, max_depth=8, random_state=rnd_st)
model.fit(train_features, train_target)
test_predictions = model.predict(test_features)
accuracy_score(test_target, test_predictions)

0.7962674961119751

Результат Random forest показывает близкие к тренировочному набору значения accuracy. Таким образом, проблем с переобучением у нашей модели нет, и её можно использовать для задачи классификации, которая выберет подходящий тариф